In [31]:
import json 
import pandas as pd 
import numpy as np 
import os 
import re 
import seaborn as sns 
import matplotlib.pyplot as plt
from omegaconf import OmegaConf 
from glob import glob 
os.chdir('../')

def load_df(log_dir):
    with open(log_dir, 'r') as f: 
        data = f.readlines()
    df = pd.DataFrame(list(pd.Series(data).map(eval).values))    
    return df 

def load_log_data(class_name, version, result_dir,method):

    df = pd.DataFrame()
    for i,v in enumerate(version):
        log_dir = os.path.join(result_dir,class_name,method,v,'log.txt')
        
        temp_df = load_df(log_dir)
        temp_df['hue'] = v 
        
        df = pd.concat([df,temp_df])
    
    return df 

def plot_log(data,class_name, metrics:list, figsize:tuple = (10,7)):
    
    fig, axes = plt.subplots(len(metrics),1,figsize=figsize)
    
    for ax, metric in zip(axes, metrics):
        sns.lineplot(
            x = 'epoch',
            y = metric,
            data = data,
            hue = data['hue'],
            ax = ax 
        )
      
    fig.suptitle(class_name)
    plt.show()
    
def read_txt(log_dir):
    with open(log_dir, 'r') as f: 
        data = f.readlines()    
    df = pd.DataFrame([eval(d) for d in data]) 
    return df 
pd.options.display.float_format = '{: .3f}'.format 

def temperature_matching(x):
    value = x.split('_')[-1]
    
    if value in ['BASELINE','focalloss']:
        return 0.05 
    elif value in ['T1','T2']:
        return float(value[-1])
    else:
        if len(value) ==1:
            return value 
        else:
            return re.search(r'T_(\d+\.\d+)', x).group(1)
    

# 전체 결과 

In [53]:
version = ['*-nb_tasks_1']

result_dir = 'results'
dataset = 'MVTecAD'
method = 'LANGCAD'
class_name = '*'
mother_dir = '/Volume/VAD/LifeLongerAD'
exp_name = '*'
seed = 0

log_dirs = glob(os.path.join(mother_dir,result_dir,method,dataset,class_name,f"{exp_name}-*",f"seed_{seed}"))
df = pd.DataFrame()

for log_dir in log_dirs:
    
    auroc_list = {}
    data = load_df(os.path.join(log_dir,'result.txt'))
    result = pd.DataFrame(data['test_metrics'][0])
    result = result.reset_index().melt(['index'])
    result.columns = ['metric','level','value']
    configs = OmegaConf.load(os.path.join(log_dir,'configs.yaml'))
    
    #configs     
    method = log_dir.split('/')[-5]
    dataset = log_dir.split('/')[-4]
    class_name = log_dir.split('/')[-3]
    exp_name = log_dir.split('/')[-2].split('-a')[0]
    
    for c in ['method','dataset','class_name','exp_name']:
        result[c] = eval(c) 
        
    for c in ['num_layers','text_embedding','sampling_ratio']:
        if c in ['num_layers','text_embedding']:
            result[c] = str(configs.MODEL.params[c])
        else:
            result[c] = configs.MODEL.params[c]            
    
    df = pd.concat([df,result])

In [ ]:
exp[
    (exp['dataset']==dataset) &
    (exp['method']=='LANGCAD') & #rand_init coreset_init
    (exp['metric']==metric) &
    (exp['level']==level) &
    ]

In [92]:
dataset = 'MVTecAD'
metric = 'auroc'
level = 'pix_level'

index = ['text_embedding']
columns = ['class_name']

exp = df 
exp1= exp[
     (exp['dataset']==dataset) &
     (exp['method']=='LANGCAD') & #rand_init coreset_init
     (exp['metric']==metric) &
     (exp['level']==level) &
     (exp['text_embedding']) &
     exp['text_embedding'].apply(lambda x : x in ['cls','pool'])
     ].pivot_table(index=index,columns=columns,values='value',aggfunc='mean')

In [93]:
exp1

class_name,bottle,cable,capsule,carpet,grid,hazelnut,leather,metal_nut,pill,screw,tile,toothbrush,transistor,wood,zipper
text_embedding,,,,,,,,,,,,,,,
cls,0.939,0.924,0.915,0.979,0.921,0.945,0.982,0.917,0.905,0.948,0.923,0.936,0.935,0.926,0.958
pool,0.947,0.938,0.921,0.981,0.932,0.956,0.984,0.925,0.904,0.954,0.940,0.938,0.937,0.934,0.954


In [94]:
temp = exp1.reset_index().melt(id_vars='text_embedding')[['class_name','value','text_embedding']]
temp.columns = ['Class','value','Method']
temp['Method'] = temp['Method'].apply(lambda x : f"ViT+{x}")

In [98]:
temp

,Class,value,Method,level
0,bottle,0.939,ViT+cls,pix level
1,bottle,0.947,ViT+pool,pix level
2,cable,0.924,ViT+cls,pix level
3,cable,0.938,ViT+pool,pix level
4,capsule,0.915,ViT+cls,pix level
5,capsule,0.921,ViT+pool,pix level
6,carpet,0.979,ViT+cls,pix level
7,carpet,0.981,ViT+pool,pix level
8,grid,0.921,ViT+cls,pix level
9,grid,0.932,ViT+pool,pix level
